<a href="https://colab.research.google.com/github/vpuzio-deloitte/digita/blob/main/digita_image_extraction_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
! pip install langchain_core langchain_google_genai boto3 botocore langchain_aws

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.1 MB/s eta 0:00:00


## Gemini

Gemini è un modello nativamente multimodale che integra le capacità di ragionamento del modello linguistico con l'elaborazione di input provenienti da diverse modalità (come testo, immagini, grafici e tabelle). Tra le sue funzionalità rientrano:

  - L'estrazione di informazioni da tabelle, grafici e figure.
  - L'identificazione di dettagli minuziosi nei dati in ingresso.
  - L'aggregazione del contesto spaziale e temporale.
  - La combinazione di informazioni provenienti da modalità differenti.

## Claude e multimodal

Anche Claude integra funzionalità multimodali

## Openai e multimodal

Openai, anche se integra in chatgpt funzioni multimodali, es. il riconoscimento di immagini, attualmente non supporta tramite le sue API funzionalità multimodali.


## Registrazione API Gemini

Quali sono i pro di gemini? Innanzitutto la versione 1.5 Flash è **gratuita** (c'è una quota max di token).

Step per registrarsi.
- Collegarsi a AI Studio https://aistudio.google.com/
- Creare una API Key


## Download immagini

Il codice scarica un'immagine da internet e la trasforma in una stringa di testo codificata in base64, rendendo più semplice la gestione o il trasferimento dell'immagine in contesti che richiedono dati testuali.

In [12]:
import base64, httpx

image_url = 'https://immagini.editorialedomani.it/version/c:ZmE4YzhhMWItZTE2OS00:ZTgzMTAzMGEtYWQ4MC00/75th-sanremo-song-festival.webp?f=3%3A2&q=0.75&w=635'
# Download and encode the image
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")


## Init LLM (Bedrock)

In [37]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm_bedrock = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


## Init LLM (Gemini)

In [33]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Initialize the model
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=userdata.get('GOOGLE_API_KEY'))


## Prompt

In [40]:
llm = llm_bedrock
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

# Inizializza il modello
llm = llm_bedrock

# Definisci il prompt template
prompt_template = PromptTemplate(
    input_variables=[],  # Se non ci sono variabili dinamiche, lascia vuoto
    template="""
        sei uno spettatore di sanremo, il classico italiano medio, un po' rozzo

        descrivimi la scena

        in napoletano, evidenzia i personaggi con bullet point
    """.strip()
)

# Crea il messaggio che include sia il testo che l'immagine
message = HumanMessage(
    content=[
        {"type": "text", "text": prompt_template.format()},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)

# Invoca il modello con il messaggio
response = llm.invoke([message])


## Display

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"### Riassunto:\n\n{response.content}"))

## Esercizio:
Costruire un output parser da cui estrapolare i soggetti dall'immagine (sceglierne una a piacere), estrarre una lista di personaggi (es. nome, descrizione, caratteristiche fisiche e psicologiche, ruolo) e con questa lista creare una storia con tutti e 3. Es. identificare protagonista, villain etc.

Hints:
- tentare di scomporre in più step
- usare PydanticOutputParser per estrarre personaggi